In [1]:
import scipy.optimize
import random
from math import exp
from math import log
from collections import defaultdict
import gzip
import numpy as np
import time
import sys

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

data = []
for l in readGz('train.json.gz'):
    data.append(l)
random.shuffle(data)

In [2]:
class KNN(object):

    def __init__(self, numUsers, numItems, lamI = 6e-2, lamJ = 6e-3, learningRate = 0.1):
        self._numUsers = numUsers
        self._numItems = numItems
        self._lamI = lamI
        self._lamJ = lamJ
        self._learningRate = learningRate
        self._users = set()
        self._items = set()
        self._Iu = defaultdict(set)
        
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def train(self, trainData, epochs=30, batchSize=500):
        
        # correlation matrix
        self.C =np.random.rand(self._numItems, self._numItems)  
        for l in xrange(self._numItems):
            self.C[l][l] = 0
            for n in xrange(l, self._numItems):
                self.C[l][n] = self.C[n][l]
              
        # change batch_size to min(batch-size, len(train))
        if len(trainData) < batchSize:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(trainData)))
            batchSize = len(trainData)
                  
        self._trainDict, self._users, self._items = self._dataPretreatment(trainData)
        N = len(trainData) * epochs
        users, pItems, nItems = self._sampling(N)
        itr = 0
        t2 = t0 = time.time()
        while (itr+1)*batchSize < N:
      
            self._mbgd(
                users[itr*batchSize: (itr+1)*batchSize],
                pItems[itr*batchSize: (itr+1)*batchSize],
                nItems[itr*batchSize: (itr+1)*batchSize]
            )
            
            itr += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.3f%% ) in %.1f seconds" %(str(itr*batchSize), 100.0 * float(itr*batchSize)/N, t2 - t0))
            sys.stderr.flush()
        if N > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %.2f samples per second\n" % (t2 - t0, N*1.0/(t2 - t0)))
            sys.stderr.flush()
            
            
    def _mbgd(self, users, pItems, nItems):
        
        prev = -2**10
        for _ in xrange(30):
            
            gradientC = defaultdict(float)
            obj = 0

            for ind in xrange(len(users)):
                u, i, j = users[ind], pItems[ind], nItems[ind]
                x_ui = sum([self.C[i][l] for l in self._Iu[u] if i != l])
                x_uj = sum([self.C[j][l] for l in self._Iu[u]])
                x_uij =  x_ui - x_uj
                
                for l in self._Iu[u]:
                    if l != i:
                        gradientC[(i,l)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[i][l]**2
                        gradientC[(l,i)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[l][i]**2
                    gradientC[(j,l)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[j][l]**2
                    gradientC[(l,j)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[l][j]**2
                    
                    obj -= 2*self._lamI * self.C[i][l]**2 + 2*self._lamJ * self.C[j][l]**2
                    
                obj += log(self.sigmoid(x_uij))
            
            #print 'OBJ: ', obj
            if prev > obj: 
                break
            prev = obj
            
            for a,b in gradientC:
                self.C[a][b] += self._learningRate * gradientC[(a,b)]
            
        #print _, '\n'
        
    def _sampling(self, N):
        
        sys.stderr.write("Generating %s random training samples\n" % str(N))
        userList = list(self._users)
        userIndex = np.random.randint(0, len(self._users), N)
        pItems, nItems = [], []
        cnt = 0
        for index in userIndex:
            u = userList[index]
            i = self._trainDict[u][np.random.randint(len(self._Iu[u]))]
            pItems.append(i)
            j = np.random.randint(self._numItems)
            while j in self._Iu[u]:
                j = np.random.randint(self._numItems)
            nItems.append(j)
            
            cnt += 1
            if not cnt %10000:
                sys.stderr.write("\rGenerated %s" %(str(cnt)))
                sys.stderr.flush()
        return userIndex, pItems, nItems

    def predictionsKNN(self, K, u):
        #slow
        if K >= self._Iu[u]:
            res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        else:
            res = []
            for i in xrange(self._numItems):
                res.append(sum(sorted([self.C[i][l] for l in self._Iu[u]], reverse=True)[:K]))
        return res

    def predictionsAll(self, u):
        
        res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        return res

    def prediction(self, u, i):
        
        scores = self.predictions(u)
        return scores[i] > sorted(scores)[self._numItem*0.8]

    def _dataPretreatment(self, data):
        dataDict = defaultdict(list)
        items = set()
        for u, i in data:
            self._Iu[u].add(i)
            dataDict[u].append(i)
            items.add(i)
        return dataDict, set(dataDict.keys()), items

In [3]:
length = len(data)
users = set()
items = set()
visited = set()
businessCount = defaultdict(int)

for l in data[:length]:
    user,business = l['userID'],l['businessID']
    users.add(user)  
    items.add(business)
    visited.add((user, business))  #visited pair
    businessCount[business] += 1
    
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count*1.0/length > 0.571: 
        break

users = list(users)
items = list(items)
Iu = defaultdict(set)
Ui = defaultdict(set)
for l in data[:length]:
    Iu[l['userID']].add(l['businessID'])
    Ui[l['businessID']].add(l['userID'])
    
users = {value:key for key, value in enumerate(users)}
items = {value:key for key, value in enumerate(items)}

In [4]:
train = [(users[l['userID']], items[l['businessID']]) for l in data[:length]]   
bpr = KNN(len(users), len(items), lamI = 4e-2, lamJ = 4e-3, learningRate = 0.1)
bpr.train(train, epochs=2, batchSize=500)

Generating 400000 random training samples
Generated 400000

OBJ:  -714.082881515
OBJ:  -468.384842843
OBJ:  -391.671572317
OBJ:  -351.099722502
OBJ:  -327.150577218
OBJ:  -312.116722227
OBJ:  -302.373538446
OBJ:  -295.976460405
OBJ:  

Processed 500 ( 0.125% ) in 1.6 seconds

-291.80315476
OBJ:  -289.169868832
OBJ:  -287.640893566
OBJ:  -286.927760733
OBJ:  -286.833161225
OBJ:  -287.218336557
13 

OBJ:  -668.410453935
OBJ:  -444.003951036
OBJ:  -359.352568517
OBJ:  -314.944098242

Processed 1000 ( 0.250% ) in 2.9 seconds

...

Processed 18500 ( 4.625% ) in 51.2 seconds

 -339.685710065
OBJ:  -322.678726406
OBJ:  -312.415402916
OBJ:  -306.094983945
OBJ:  -302.257105587
OBJ:  -300.074773864
OBJ:  -299.048165889
OBJ:  -298.859235653
OBJ:  -299.297063924
11 

OBJ:  -656.399708032
OBJ:  -435.147345861
OBJ:  -376.379396893
OBJ:  -347.771718398

Processed 19000 ( 4.750% ) in 52.6 seconds


OBJ:  -331.398323495
OBJ:  -321.479330678
OBJ:  -315.357289177
OBJ:  -311.641310109
OBJ:  -309.537692451
OBJ:  -308.563598029
OBJ:  -308.410353965
OBJ:  -308.872623558
11 



KeyboardInterrupt: 

In [ ]:
predictions = open("predictions_Visit.txt", 'w')
cnt = 0
for l in open("pairs_Visit.txt"):
    cnt += 1
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u not in users:  #no history of user, use popularity
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
        continue
        
    if i not in items:   #no one visited before
        predictions.write(u + '-' + i + ",0\n")
        continue
        
    #scores = bpr.predictionsKNN(10, users[u])
    scores = bpr.predictionsAll(users[u])
    #thrhd = sorted(scores)[len(items)*7/10]
    score = scores[items[i]]
    
    num = len([1 for x in scores if x > score])

    #if score >= thrhd:
    if num < len(scores)*30/100:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
        
    sys.stderr.write("\rProcessed %s" %(str(cnt)))
    sys.stderr.flush()
predictions.close()

In [20]:
cnt = 0
for l in open("predictions_Visit.txt", 'r'):
    if l.startswith("userID"):
    #header
        continue
    _, y = l.strip().split(',')
    if y == '1':
        cnt += 1
cnt

18644

In [19]:
# predict with users' thrhd
predictions = open("predictions_Visit.txt", 'w')
cnt = 0
for l in open("pairs_Visit.txt"):
    cnt += 1
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u not in users:  #no history of user, use popularity
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
        continue
        
    if i not in items:   #no one visited before
        predictions.write(u + '-' + i + ",0\n")
        continue
        
    scores = pred(users[u])
    thrhd = min([scores[items[it]] for it in Iu[u]])
    
    if scores[items[i]] >= thrhd:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
        
    sys.stderr.write("\rProcessed %s" %(str(cnt)))
    sys.stderr.flush()
predictions.close()

Processed 40001

In [ ]:
from theano_bpr import BPR
import theano
import theano.tensor as T
from theano import function, config, shared, sandbox  

In [ ]:
length = len(data)*10/11
users = set()
items = set()
visited = set()
businessCount = defaultdict(int)

for l in data[:length]:
    user,business = l['userID'],l['businessID']
    users.add(user)  
    items.add(business)
    visited.add((user, business))  #visited pair
    businessCount[business] += 1
    
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count*1.0/length > 0.571: 
        break

users = list(users)
items = list(items)
Iu = defaultdict(set)
Ui = defaultdict(set)
for l in data[:length]:
    Iu[l['userID']].add(l['businessID'])
    Ui[l['businessID']].add(l['userID'])
    
unvisited = set()
while len(unvisited) < len(data)-length:    
    user = users[random.randint(0,len(users)-1)]
    item = items[random.randint(0,len(items)-1)]
    if item not in Iu[user]:
        unvisited.add((user, item))
    
users = {value:key for key, value in enumerate(users)}
items = {value:key for key, value in enumerate(items)}

In [ ]:
from theano_bpr import BPR
train = [(users[l['userID']], items[l['businessID']]) for l in data[:length]]   
for U in [1e-4,1e-3,0.1]:
    for I in [1e-4,1e-3,0.1]:
        for J in [1e-4,1e-3,0.1]:
            for bias in [0.1,1,3]:
                bpr = BPR(5, len(users), len(items), lambda_u = U, lambda_i = I, lambda_j = J, lambda_bias = bias)
                bpr.train(train, epochs=150, batch_size=1000)
                
                print U,I,J,bias
                cnt = 0
                for l in data[length:]:
                    u,i = l['userID'], l['businessID']
                    if u not in users:  #no history of user, use popularity
                        if i in return1:
                            cnt += 1
                        continue

                    if i not in items:   #no one visited before
                        continue

                    scores = bpr.predictions(users[u])
                    thrhd = min([scores[items[it]] for it in Iu[u]])

                    if scores[items[i]] >= thrhd:
                        cnt += 1
                ac = cnt*1.0/(len(data)-length)
                print "acr posi: " + str(ac),
                cnt = 0
                for u,i in unvisited:
                    
                    if u not in users:  #no history of user, use popularity
                        if i not in return1:
                            cnt += 1
                        continue

                    if i not in items:   #no one visited before
                        cnt += 1
                        continue

                    scores = bpr.predictions(users[u])
                    thrhd = min([scores[items[it]] for it in Iu[u]])

                    if scores[items[i]] < thrhd:
                        cnt += 1
                print "acr nega: " + str(cnt*1.0/len(unvisited)),
                print 'avr: ' + str((ac+cnt*1.0/len(unvisited))/2.0)